# **Stock Price Prediction using LSTM**


### Problem Statement
Financial markets are volatile and complex, making accurate stock price prediction challenging.  
This project aims to predict future stock closing prices using historical data and deep learning techniques.

### Objective
- Predict future stock prices using historical trends  
- Capture temporal dependencies using LSTM  
- Evaluate model performance using appropriate metrics  

### Real-World Relevance & Motivation
- Supports data-driven investment decisions  
- Helps in trend identification and risk analysis  
- Demonstrates real-world application of AI in finance

Dataset source: Public dataset collected from kaggle as on 27 Dec 2025


# **Exploratory Data Analysis and Data Pre Processing**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
df = pd.read_csv("NIFTY_50_COMPANIES.csv")
df.head()

Loaded the dataset using pandas and exploring dataset

In [ ]:
df = df[df['Ticker'] == 'RELIANCE.NS']

Only Reliance stock is selected in the ticker as the sock prediction model is for a single stock

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date')
df.set_index('Date', inplace=True)

Converted the date column to datetime format

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

No missing values for the relevant data used in model

In [ ]:
plt.figure(figsize=(12,5))
plt.plot(df['Close'])
plt.title("Closing Price Trend")
plt.xlabel("Date")
plt.ylabel("Price")
plt.show()

Closing price long term trend exhibits sequential pattern which is perfect for forecasting time-series data

In [ ]:
df['Daily_Return'] = df['Close'].pct_change()
plt.figure(figsize=(10,4))
plt.plot(df['Daily_Return'])
plt.title("Daily Returns")
plt.show()

Daily returns over the years are almost linear. The spikes are due to extreme events like stock split. We ignore these data and proceed ahead.

In [ ]:
df['MA20'] = df['Close'].rolling(20).mean()
df['MA50'] = df['Close'].rolling(50).mean()
plt.figure(figsize=(12,5))
plt.plot(df['Close'], label='Close')
plt.plot(df['MA20'], label='20-Day MA')
plt.plot(df['MA50'], label='50-Day MA')
plt.legend()
plt.title("Moving Average Analysis")
plt.show()

The moving averages smooth price fluctuations which helps in analysing short and medium term trends. This confirms the bullish market in the long run

In [ ]:
plt.figure(figsize=(6,4))
sns.heatmap(
    df[['Open','High','Low','Close','Volume']].corr(),
    annot=True,
    cmap='coolwarm'
)
plt.title("Feature Correlation Heatmap")
plt.show()

Stock price related data are highly correlated. Focus on closing price to avoid redundancy.

In [ ]:
sns.boxplot(data=df[['Open','High','Low','Close']])
plt.title("Price Distribution")
plt.show()

The price distribution shows outliers in the data over time but cannot ignore them as it is crucial for prediction.

# **Feature Engineering**

In [ ]:
data = df[['Open', 'High', 'Low', 'Close', 'Volume']]
data = data.values

Used only relavant data for prediction

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

Normalised data using Min-Max scaling

In [ ]:
window_size=60
X, y = [], []

for i in range(window_size, len(scaled_data)):
    X.append(scaled_data[i - window_size : i])
    y.append(scaled_data[i, 3])

X = np.array(X)
y = np.array(y)

In [ ]:
split = int(0.8 * len(X))

X_train = X[:split]
X_test  = X[split:]

y_train = y[:split]
y_test  = y[split:]

In [ ]:
WINDOW_SIZES = [60]
LSTM_UNITS = [32, 50]
BATCH_SIZES = [32]
EPOCHS = 10

In [ ]:
def build_lstm_model(units, input_shape):
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(LSTM(units, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

def create_sequences(data, window_size):
    X, y = [], []
    for i in range(window_size, len(data)):
        X.append(data[i-window_size:i])
        y.append(data[i, 3])
    return np.array(X), np.array(y)

Used 60 days as data framework sequences for more reliable long term pattern learning

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
results = []

for window in WINDOW_SIZES:
    X, y = create_sequences(scaled_data, window)

    split = int(0.8 * len(X))
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]

    for units in LSTM_UNITS:
        for batch in BATCH_SIZES:

            model = build_lstm_model(
                units,
                input_shape=(X_train.shape[1], X_train.shape[2])
            )

            history = model.fit(
                X_train, y_train,
                epochs=20,
                batch_size=batch,
                validation_split=0.1,
                verbose=0
            )

            val_loss = min(history.history['val_loss'])

            results.append({
                'window': window,
                'units': units,
                'batch': batch,
                'val_loss': val_loss
            })

Data is split chronologicaly for accurate training and testing. Trained multiple LSTM configurations and selected the best model based on validation loss

In [ ]:

from tensorflow.keras.layers import LSTM, Dense, Dropout, Input

model = Sequential()

model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))

model.add(LSTM(50, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(50))
model.add(Dropout(0.2))

model.add(Dense(1))

Used a stacked LSTM with dropout to capture long-term dependencies while preventing overfitting.

In [ ]:
model.compile(
    optimizer='adam',
    loss='mean_squared_error'
)

Adam is used for fast, stable convergence with adaptive learning rates, and MSE is used because it strongly penalizes large errors, making it suitable for regression-based price prediction.

In [ ]:
history = model.fit(
    X_train,
    y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.1
)

 The model was trained for 20 epochs with a batch size of 32 and a 10% validation split.Training loss consistently decreases. Validation loss remains low and stable. However minor fluctuations in validation loss are expected due to market volatility and noise.

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training vs Validation Loss')
plt.legend()
plt.show()

The loss curves show stable learning without overfitting, indicating good model generalization

In [ ]:
predicted = model.predict(X_test)

Used the trained model to predict stock prices on unseen test data

In [ ]:
dummy_pred = np.zeros((len(predicted), 5))
dummy_pred[:, 3] = predicted[:, 0]

predicted_prices = scaler.inverse_transform(dummy_pred)[:, 3]

dummy_actual = np.zeros((len(y_test), 5))
dummy_actual[:, 3] = y_test

actual_prices = scaler.inverse_transform(dummy_actual)[:, 3]

Inverse scaling the predicted closing prices to its actual values

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(actual_prices, label="Actual Price")
plt.plot(predicted_prices, label="Predicted Price")
plt.legend()
plt.title("LSTM Stock Price Prediction")
plt.show()

There is a close match between predicted price and the actual price

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

rmse = np.sqrt(mean_squared_error(actual_prices, predicted_prices))
mae = mean_absolute_error(actual_prices, predicted_prices)

print("RMSE:", rmse)
print("MAE:", mae)

Lower values of MAE and RMSE indicate better predictive performance, and these results suggest that the LSTM model performs pretty well

In [ ]:
n_days = 1

last_window = scaled_data[-window_size:]

last_window = last_window.reshape(1, window_size, scaled_data.shape[1])


next_day_scaled = model.predict(last_window)


dummy = np.zeros((1, scaled_data.shape[1]))
dummy[0, 3] = next_day_scaled[0, 0]

next_day_price = scaler.inverse_transform(dummy)[0, 3]

print("Predicted next day closing price:", next_day_price)

Using the last 60 days of data, the LSTM predicts the next day’s closing price after inverse scaling.

In [ ]:
def predict_future_days(model, scaled_data, window_size, n_days):
    future_predictions = []

    current_window = scaled_data[-window_size:].copy()

    for _ in range(n_days):
        X = current_window.reshape(1, window_size, scaled_data.shape[1])
        next_scaled = model.predict(X)[0, 0]

        future_predictions.append(next_scaled)

        next_row = current_window[-1].copy()
        next_row[3] = next_scaled
        current_window = np.vstack([current_window[1:], next_row])

    return future_predictions

n_days = 10
future_scaled = predict_future_days(model, scaled_data, window_size, n_days)


future_prices = []
for val in future_scaled:
    dummy = np.zeros((1, scaled_data.shape[1]))
    dummy[0, 3] = val
    future_prices.append(scaler.inverse_transform(dummy)[0, 3])

print("Future predicted prices:")
for i, price in enumerate(future_prices, 1):
    print(f"Day {i}: {price}")

Used recursive forecasting where each predicted day is fed back as input to predict multiple future days

# **Evaluation and Analysis**

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

rmse = np.sqrt(mean_squared_error(actual_prices, predicted_prices))
mae = mean_absolute_error(actual_prices, predicted_prices)
mape = np.mean(np.abs((actual_prices - predicted_prices) / actual_prices)) * 100
r2 = r2_score(actual_prices, predicted_prices)

direction_actual = np.sign(np.diff(actual_prices))
direction_pred = np.sign(np.diff(predicted_prices))
directional_accuracy = np.mean(direction_actual == direction_pred) * 100

print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"MAPE: {mape}%")
print(f"R² Score: {r2}")
print(f"Directional Accuracy: {directional_accuracy}%")

MAE is robust to outliers, indicating consistent prediction accuracy across most periods. Given that the stock price ranges between ₹400 and ₹1600, this error is relatively small.

RMSE being higher than MAE indicates the presence of occasional larger errors, typically during high-volatility periods. This is expected in financial time-series due to sudden market movements and external shocks.

In financial forecasting, a MAPE below 5% is generally considered strong performance.

An R² of 0.98 indicates a very strong fit and confirms that the LSTM has effectively learned long-term price patterns.

A directional accuracy close to 50% suggests performance similar to random guessing for short-term direction. Reasons for Low Directional Accuracy might be that the stock prices are influenced by news, sentiment, macroeconomic events, and sudden shocks.

# **AI and Biasness**

The absence of external factors such as macroeconomic indicators, company fundamentals, or news sentiment introduces representation bias, as the model captures only technical price movements. Evaluation metrics focused on error reduction may also mask directional or risk-related biases, giving an overly optimistic view of model performance.

# **Conclusion**

This project successfully demonstrates the application of an LSTM-based deep learning model for stock price prediction using historical market data. The model was able to capture temporal patterns and trends in stock prices, producing reasonably accurate forecasts based on past price movements. Data preprocessing, feature scaling, and sequence modeling played a critical role in improving model performance.

Overall, the project highlights the potential of LSTM networks as effective tools for time-series forecasting in financial markets. While the model provides useful predictive insights, its outputs are best suited for decision support rather than standalone investment decisions, especially in dynamic market conditions. The project also establishes a strong foundation for future enhancements through additional data sources and advanced evaluation techniques.

# **Future Scope**

The project can be further enhanced by incorporating additional data sources such as macroeconomic indicators, company fundamentals, and news sentiment to improve predictive accuracy. Advanced deep learning architectures like GRU, CNN-LSTM hybrids, or attention-based models can be explored to better capture complex market dynamics.

Future work may also include multi-stock or portfolio-level prediction, improved evaluation using directional and risk-based metrics, and real-time model retraining to adapt to changing market conditions. Integrating the model into a decision-support system for trading or risk analysis would increase its practical applicability.